In [2]:
import argparse
import json
import os
import sys
import traceback
from glob import glob
import math
import random
import numpy as np
from dotenv import load_dotenv
from azureml.core import Dataset, Datastore, Experiment, Run, Workspace
from azureml.core.authentication import AzureCliAuthentication
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from dotnetcore2 import runtime
runtime.version = ("18", "10", "0")

In [9]:
    #cli_auth = AzureCliAuthentication()
    
    #Create folder if not exist
    train_test_data_folder = os.path.join(os.getcwd(), 'data/tmp/train_test_data')
    os.makedirs(train_test_data_folder, exist_ok=True)

    #Connect to workspace
    ws = Workspace.from_config('config.json')

    #Get data from datastore
    datastore = Datastore(ws)
    dataset = Dataset.get_by_name(ws, name='ratings')
    dataset = dataset.to_pandas_dataframe()
    dataset

In [ ]:
# Formatting the dataset.
dataset= dataset.rename({"Column1": "reviewer", "Column2": "movie", "Column3": "score"},axis="columns")
dataset= dataset.pivot(index="reviewer",columns="movie",values="score").fillna(0).astype(int)
i=1
for index in dataset.columns:
    while(index!=i):
        dataset.insert(i,i,[0]*6040)
        i+=1
    i+=1
dataset.add_prefix('movie')

/home/student/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3364: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  if (await self.run_code(code, result,  async_=asy)):


movie,movie1,movie2,movie3,movie4,movie5,movie6,movie7,movie8,movie9,movie10,...,movie3943,movie3944,movie3945,movie3946,movie3947,movie3948,movie3949,movie3950,movie3951,movie3952
reviewer,,,,,,,,,,,,,,,,,,,,,
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0,0,0,2,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6037,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6038,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Compile the trainig set and test set
import random
dataset_test = dataset 
dataset_train = dataset
for index, row in dataset_train.iterrows():
    non_null=row.to_numpy().nonzero()[0]
    indexes = random.sample(range(0,len(row.to_numpy().nonzero()[0])-1), 10)
    for i in indexes:
        dataset_train.iat[index-1,non_null[i]]=0
np.save(f"{train_test_data_folder}/{dataset_test}.npy", dataset_test)
np.save(f"{train_test_data_folder}/{dataset_train}.npy", dataset_train)
dataset_train

ValueError: Sample larger than population or is negative

In [ ]:
datastore.upload(src_dir=train_test_data_folder, target_path='imdb_train_test',overwrite=True)
train_test_data = Dataset.File.from_files([(datastore, 'imdb_train_test')],validate=False)
train_test_data.register(
workspace=ws,
name="imdb_train_test",
description="Processed train- and testdata for consumption by model",
create_new_version=True)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 0 files
Uploaded 0 files


{
  "source": [
    "('workspaceblobstore', 'imdb_train_test')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "acca1d3e-4f73-48d1-97cc-0e64a0a3714f",
    "name": "imdb_train_test",
    "version": 1,
    "description": "Processed train- and testdata for consumption by model",
    "workspace": "Workspace.create(name='imdbML-jazi-yacine', subscription_id='350ff211-0872-40d5-a629-9b672edbe452', resource_group='imdb-jazi-yacine')"
  }
}